In [ ]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
import random
from math import radians, cos, sin, asin, sqrt
import names

GEOJSONPATH = "./map.geojson"
 # Time taken to traverse x haversive = x*scale seconds

# Data Generation

In [ ]:
nm = folium.Map()
folium.GeoJson("./map.geojson").add_to(nm)
nm

## Loading Data

In [ ]:
gdf = gpd.read_file(GEOJSONPATH)
X_coord = gdf.centroid.x.to_numpy()[0:150]
Y_coord = gdf.centroid.y.to_numpy()[0:150]

In [ ]:

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [ ]:
savearr = []
distMatrix = []

for j in range(0, len(X_coord)):
    dist = []
    for i in range(0,len(X_coord)):
        if i!= j:
            distance = haversine(X_coord[j], Y_coord[j], X_coord[i], Y_coord[i])
            dist.append((distance,i))

    sortedarr = sorted(dist, 
       key=lambda x: x[0])
    distMatrix.append(dist)
    savearr.append(sortedarr)

np.save("onearr", savearr)
np.save("adjmatrix", distMatrix)

In [ ]:
sortedMatrix = np.load("./onearr.npy")
adjMatrix = np.load("./adjmatrix.npy")
adjList = {}

for i in range(0, len(X_coord)):
    nodesConnected = sortedMatrix[i][0:6]
    adjList[i] = nodesConnected


## Algorithms and Classes

In [ ]:
from queue import PriorityQueue

def djikstraPath(start, end, adjList):
    pq = PriorityQueue()
    pq.put((0,start,-1))
    visited = set()
    path = []
    parent = {}
    parent[start] = -1

    while not pq.empty():
        top = pq.get()

        dist = top[0]
        node = int(top[1])
        prev = int(top[2])

        parent[node] = prev

        if node == end:
            break
        if node in visited:
            continue

        visited.add(node)
      

        for child in adjList[node]:
            if int(child[1]) not in visited:
                pq.put((dist + child[0], int(child[1]), node))
    
    node = end

    while parent[node] != -1:
        path.append(node)
        node = parent[node]

    path.reverse()
    return path

        
def createRandomRoute(start, end, adjList, addDetourProb = 0.5, maxDetours = 2, detourHops = 1):

    route = [start]
    possibleDetours = [int(i[1]) for i in adjList[start]]

    for i in range(0,detourHops):
        for j in possibleDetours:
            addedDetours = []
            for k in adjList[j]:
                addedDetours.append(int(k[1]))
            possibleDetours = possibleDetours + addedDetours

    possibleDetours = [*set(possibleDetours)]

    for i in range(0, maxDetours):

        if random.random() < addDetourProb:
            route.append(random.choice(possibleDetours))
        
        addDetourProb = addDetourProb/2

    route.append(end)
    # print("Route: ", route)
    path = [start]
    for i in range(1, len(route)):
        path = path + djikstraPath(route[i-1],route[i], adjList)
    
    # print(path)
    return path

def createRandomRouteQuick(start, end, adjList, addDetourProb = 0.5, maxDetours = 2, detourHops = 1):
    route = [start]
    possibleDetours = [int(i[1]) for i in adjList[start]]

    for i in range(0,detourHops):
        for j in possibleDetours:
            addedDetours = []
            for k in adjList[j]:
                addedDetours.append(int(k[1]))
            possibleDetours = possibleDetours + addedDetours

    possibleDetours = [*set(possibleDetours)]

    for i in range(0, maxDetours):

        if random.random() < addDetourProb:
            route.append(random.choice(possibleDetours))
        
        addDetourProb = addDetourProb/2

    route.append(end)

    return route

In [ ]:
RANDOM_REST = 5
FIRST_LEG_REST_L = 5
FIRST_LEG_REST_R = 7
SECOND_LEG_REST_L = 10
SECOND_LEG_REST_R = 14
HAVERSINE_SCALE = 30
LAPTOPS = ["Infinix InBook", "Acer Aspire 5 A515-51G", "Dell Inspiron 3567","Asus VivoBook", "Lenovo ThinkPad E480", "Dell Alienware m15 R7"]
LAPTOP_OS = ["Windows", "Linux"]
MOBILES = ["Google Pixel 7", "IPhone 14", "IPhone 13", "Samsung galaxy S22", "vivo X80", "Samsung galaxy Z Flip", "OnePlus 10"]
MOBILE_OS = ["Android", "IoS"]

REQUEST_PROB_TRAVELLING = 0.2 # Probability that user sends req to router while travelling (not resting)
REQUEST_PROB_RESTING = 0.9

from random_word import RandomWords
random_word_gen = RandomWords()


class Person :

    def __init__(self,_name, _id, _start, _end):
        self.name = _name
        self.id = _id
        self.start_loc = _start
        self.end_loc = _end 
        self.current_loc = 0
        self.travel_time = 0
        self.current_city = _start
        self.started = False
        self.start_time = random.randint(1,RANDOM_REST)
        self.journey_parity = 0
        self.laptop = random.choice(LAPTOPS)
        self.laptop_os = random.choices(LAPTOP_OS)
        self.mobile = random.choice(MOBILES)
        self.mobile_os = random.choice(MOBILE_OS)

    def _get_random_request(self, router, resting=True):
        request = [router]
        if resting:
            request.append(self.laptop)
            request.append(self.laptop_os)
        else:
            request.append(self.mobile)
            request.append(self.mobile_os)

        request.append("https://www." + random_word_gen.get_random_word() + ".com")
        request.append(random.randint(1000,100000))

        return request

    def advance(self):
        if self.started:
            # Travelling

            if self.current_loc < len(self.route)-1:
                self.travel_time = self.travel_time + 1
                lat1 = X_coord[self.route[self.current_loc]]
                lon1 = Y_coord[self.route[self.current_loc]]
                lat2 = X_coord[self.route[self.current_loc+1]]
                lon2 = Y_coord[self.route[self.current_loc+1]]
                if int(haversine(lat1,lon1,lat2,lon2) * HAVERSINE_SCALE) <= self.travel_time:
                    self.travel_time = 0
                    self.current_loc = self.current_loc + 1
                    self.current_city = self.route[self.current_loc]
            else:
                self.journey_parity = (self.journey_parity ^ 1)
                self.travel_time = 0
                self.started = False
                self.current_loc = 0
                self.start_loc , self.end_loc = self.end_loc,self.start_loc
                self.current_city = self.start_loc
                if self.journey_parity == 1:
                    self.start_time = random.randint(FIRST_LEG_REST_L,FIRST_LEG_REST_R)
                else:
                    self.start_time = random.randint(SECOND_LEG_REST_L,SECOND_LEG_REST_R)

            if random.random() < REQUEST_PROB_TRAVELLING:
                request = self._get_random_request(self.current_city,resting=False)
                routers[self.current_city].get_request(request[-1])
                return request
            else:
                return []
        else:
            # Resting
            
            if self.start_time <= 0:
                self.route = createRandomRouteQuick(self.start_loc,self.end_loc,adjList, detourHops=3)
                self.started = True
            else:
                self.start_time = self.start_time-1

            if random.random() < REQUEST_PROB_RESTING:
                request = self._get_random_request(self.current_city,resting=True)
                routers[self.current_city].get_request(request[-1])
                return request
                
            else:
                return []


In [ ]:
import random


OUTPUT_RATE = 10 # Packets per time step
PACKET_SIZE = 4096 # In bytes
OUTPUT_RATE_VARIANCE = 2
QUEUE_SIZE = 100 # Packets in queue
QUEUE_VARIANCE = 20
DEF_TEMP = 30
MAX_TEMP = 70

class Router:
    def __init__(self, id, x_loc, y_loc):
        self.id = id
        self.x_loc = x_loc
        self.y_loc = y_loc
        self.output_rate = random.randint(OUTPUT_RATE - OUTPUT_RATE_VARIANCE, OUTPUT_RATE + OUTPUT_RATE_VARIANCE)
        self.queue_size = random.randint(QUEUE_SIZE - QUEUE_VARIANCE, QUEUE_SIZE + QUEUE_VARIANCE)
        self.queue_filled = 0
        self.temperature = DEF_TEMP
        self.packets_served = 0

    def get_request(self,request_size):
        new_packets = request_size // PACKET_SIZE

        if self.queue_filled + new_packets <= self.queue_size:
            self.queue_filled = self.queue_filled + new_packets


    def advance(self):
        self.packets_served = self.packets_served + min(self.queue_filled, self.output_rate)
        self.queue_filled = max(0, self.queue_filled - self.output_rate)
        self.temperature = DEF_TEMP + (MAX_TEMP-DEF_TEMP)*(self.queue_filled/self.queue_size)


In [ ]:
import random
class Spawner:
    def __init__(self):
        self.global_id = 1
        
    def spawn(self):
        spawned = Person(names.get_full_name(),self.global_id,random.randint(1,len(X_coord)-1),random.randint(1,len(X_coord)-1))
        self.global_id = self.global_id + 1
        return spawned
    
    def generate_population(self, population_size):
        population = []
        for person_number in range(0,population_size):
            population.append(self.spawn())

        return population

## Generating Population and routers

In [ ]:
spawner = Spawner()
pop = spawner.generate_population(200)

In [ ]:
routers = []
for i in range(0,len(X_coord)):
    router = Router(i, X_coord[i],Y_coord[i])
    routers.append(router)

## Simulation

In [ ]:
import time
import csv

TIME = 3000

current_time = int(time.time())

# print("CURRENT TIME IS ", current_time)

PERSON_DETAILS = "people.csv"
PERSON_FIELDS = ["ID", "Name"]

person_file = open(PERSON_DETAILS, 'w')
person_writer = csv.writer(person_file)
person_writer.writerow(PERSON_FIELDS)

for person in pop:
    row = [person.id, person.name]
    person_writer.writerow(row)
person_file.close()

ROUTER_DETAILS = "router.csv"
ROUTER_FIELDS = ["ID", "X_loc", "Y_loc"]

router_file = open(ROUTER_DETAILS, 'w')
router_writer = csv.writer(router_file)
router_writer.writerow(ROUTER_FIELDS)

for router in routers:
    row = [router.id, router.x_loc,router.y_loc]
    router_writer.writerow(row)

router_file.close()

REQUEST_DETAILS = "request.csv"
REQUEST_FIELDS = ["Timestamp", "Person_ID", "Router_ID", "Device", "OS","Req_resource","Req_data"]

req_file = open(REQUEST_DETAILS, 'w')
req_writer = csv.writer(req_file)
req_writer.writerow(REQUEST_FIELDS)

ROUTER_DIAG_DETAILS = "diagnostics.csv"
ROUTER_DIAG_FIELDS = ["Timestamp", "Router_ID", "Queued_packets","Packets_served","Temperature"]

diag_file = open(ROUTER_DIAG_DETAILS, 'w')
diag_writer = csv.writer(diag_file)
diag_writer.writerow(ROUTER_DIAG_FIELDS)

WRITE_AFTER = 300
counter = 0
request_rows = []
diag_rows = []

for timestamp in range(current_time, current_time + TIME):

    if counter >= WRITE_AFTER:
        counter = 0
        req_writer.writerows(request_rows)
        diag_writer.writerows(diag_rows)
        request_rows = []
        diag_rows = []
    
    for person in pop:
        request = person.advance()

        if len(request) > 0:
            request = [timestamp, person.id] + request
            request_rows.append(request)
            # req_writer.writerow(request)

    for router in routers:
        router.advance()

        diag = [timestamp, router.id,router.queue_filled,router.packets_served,router.temperature]
        diag_rows.append(diag)
        # diag_writer.writerow(diag)
    
    counter = counter + 1

diag_file.close()
req_file.close()